# Homework 4: Neural Sequence Labeling

**Due March 4, 2020 at 11:59PM**


In this homework, you will be implementing, training, and evaluating an LSTM for part-of-speech tagging using the PyTorch library.

**Before beginning, please switch your Colab session to a GPU runtime** 

Go to Runtime > Change runtime type > Hardware accelerator > GPU

### Setup

In [0]:
# import libraries
import torch
import numpy as np
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

In [59]:
# if this cell prints "Running on cpu", you must switch runtime environments
# go to Runtime > Change runtime type > Hardware accelerator > GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cuda


### Download & Load Pretrained Embeddings

In this assignment, we will be using GloVe pretrained word embeddings. You can read more about GloVe here: https://nlp.stanford.edu/projects/glove/

**Note**: this section will take *several minutes*, since the embedding files are large. Files in Colab may be cached between sessions, so you may or may not need to redownload the files each time you reconnect. 


In [61]:
# download pretrained word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-03-04 22:01:03--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-03-04 22:01:03--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-03-04 22:01:03--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [0]:
def read_embeddings(filename, vocab_size=10000):
  """
  Utility function, loads in the `vocab_size` most common embeddings from `filename`
  
  Arguments:
  - filename:     path to file
                  automatically infers correct embedding dimension from filename
  - vocab_size:   maximum number of embeddings to load

  Returns 
  - embeddings:   torch.FloatTensor matrix of size (vocab_size x word_embedding_dim)
  - vocab:        dictionary mapping word (str) to index (int) in embedding matrix
  """

  # get the embedding size from the first embedding
  with open(filename, encoding="utf-8") as file:
    word_embedding_dim = len(file.readline().split(" ")) - 1

  vocab = {}

  embeddings = np.zeros((vocab_size, word_embedding_dim))

  with open(filename, encoding="utf-8") as file:
    for idx, line in enumerate(file):

      if idx + 2 >= vocab_size:
        break

      cols = line.rstrip().split(" ")
      val = np.array(cols[1:])
      word = cols[0]
      embeddings[idx + 2] = val
      vocab[word] = idx + 2
  
  # a FloatTensor is a multidimensional matrix
  # that contains 32-bit floats in every entry
  # https://pytorch.org/docs/stable/tensors.html
  return torch.FloatTensor(embeddings), vocab

Running the cell below lists all the files in the current directory. 

In [5]:
!ls -lh

total 2.9G
-rw-rw-r-- 1 root root 332M Aug  4  2014 glove.6B.100d.txt
-rw-rw-r-- 1 root root 662M Aug  4  2014 glove.6B.200d.txt
-rw-rw-r-- 1 root root 990M Aug 27  2014 glove.6B.300d.txt
-rw-rw-r-- 1 root root 164M Aug  4  2014 glove.6B.50d.txt
-rw-r--r-- 1 root root 823M Oct 25  2015 glove.6B.zip
drwxr-xr-x 1 root root 4.0K Mar  3 18:11 sample_data


You should see several embedding files, which are all formatted as

```
glove.6B.<emb_dim>d.txt
```

Each `txt` file contains `emb_dim` dimensional embeddings for 400,000 unique, uncased words. The script below loads the `vocab_size` most common words from the embedding file into a matrix we can give to our model. All other words will later be mapped to the `UNKNOWN` embedding.

In [0]:
# this loads the 10,000 most common word 50-dimensional embeddings
vocab_size = 10000
embeddings, vocab = read_embeddings('glove.6B.50d.txt', vocab_size)

## Part 1: Batching the data

Implement the `get_batches` function in the `Dataset` class below. 

**Please make sure that**

*   Your implementation is self-contained. That is, all helper functions and variables are defined within `get_batches`.
*   Your implementation can handle variable batch sizes. You may not assume that the value with always be 32



In [0]:
class Dataset():
  def __init__(self, filename, is_labeled):
    self.is_labeled = is_labeled
    # if the file is not labeled, the Dataset has no tags (see read_data)
    if is_labeled:
      self.sentences, self.tags = self.read_data(filename, is_labeled)
    else:
      self.sentences = self.read_data(filename, is_labeled)
      self.tags = None

  def read_data(self, filename, is_labeled):
    """
    Utility function, loads text file into a list of sentence and tag strings

    Arguments:
    - filename:     path to file
    - is_labeled:   whether the file contains tags for each word or not
        > if True, we assume each line is formatted as "<word>\t<tag>\n"
        > if False, we assume each line is formatted as "<word>\n"

    Returns:
    - sentences:    a list of sentences, where each sentence is a list 
                    words (strings)

    if is_labeled=True, also returns
    - tags:         a list of tags for each sentence, where tags[i] contains
                    a list of tags (strings) that correspond to the words in 
                    sentences[i]
    """
    sentences = []
    tags = []

    current_sentence = []
    current_tags = []

    with open(filename, encoding='utf8') as f:
      # iterate over the lines in the file
      for line in f:
        if len(line) == 0:
          continue
        if line == '\n':
          if len(current_sentence) != 0:
            sentences.append(current_sentence)
            tags.append(current_tags)

          current_sentence = []
          current_tags = []
        else:
          if is_labeled:
            columns = line.rstrip().split('\t')
            word = columns[0].lower()
            tag = columns[1]

            current_sentence.append(word)
            current_tags.append(tag)
          else:
            column = line.rstrip().split('\t')
            word = column[0].lower()
            current_sentence.append(word)
      
      if is_labeled:
        return sentences, tags
      else:
        return sentences

  def get_batches(self, batch_size, vocab, tagset):
    """

    Batches the data into mini-batches of size `batch_size`

    Arguments:
    - batch_size:       the desired output batch size
    - vocab:            a dictionary mapping word strings to indices
    - tagset:           a dictionary mapping tag strings to indices

    Outputs:

    if is_labeled=True:
    - batched_word_indices:     a list of matrices of dimension (batch_size x max_seq_len)
    - batched_tag_indices:      a list of matrices of dimension (batch_size x max_seq_len)
    - batched_lengths:          a list of arrays of length (batch_size)

    if is_labeled=False:
    - batched_word_indices:     a list of matrices of dimension (batch_size x max_seq_len)
    - batched_lengths:          a list of arrays of length (batch_size)


    Description: 

    This function partitions the data into batches of size batch_size. If the number
    of sentences in the document is not an even multiple of batch_size, the final batch
    will contain the remaining elements. For example, if there are 82 sentences in the 
    dataset and batch_size=32, we return a list containing two batches of size 32 
    and one final batch of size 18.

    batched_word_indices[b] is a (batch_size x max_seq_len) matrix of integers, 
    containing index representations for sentences in the b-th batch in the document. 
    The `vocab` dictionary provides the correct mapping from word strings to indices. 
    If a word is not in the vocabulary, it gets mapped to UNKNOWN_INDEX (1).
    `max_seq_len` is the maximum sentence length among the sentences in the current batch, 
    which will vary between different batches. All sentences shorter than max_seq_len 
    should be padded on the right with PAD_INDEX (0).

    If the document is labeled, we also batch the document's tags. Analogous to 
    batched_word_indices, batched_tag_indices[b] contains the index representation
    for the tags corresponding to the sentences in the b-th batch  in the document. 
    The `tagset` dictionary provides the correct mapping from tag strings to indicies. 
    All tag lists shorter than `max_seq_len` are padded with IGNORE_TAG_INDEX (-100).

    batched_lengths[b] is a vector of length (batch_size). batched_lengths[b][i] 
    contains the original sentence length *before* padding for the i-th sentence
    in the currrent batch. 

    """
    PAD_INDEX = 0             # reserved for padding words
    UNKNOWN_INDEX = 1         # reserved for unknown words
    IGNORE_TAG_INDEX = -100   # reserved for padding tags

    # randomly shuffle the data
    np.random.seed(159) # DON'T CHANGE THIS
    shuffle = np.random.permutation(range(len(self.sentences)))

    sentences = [self.sentences[i] for i in shuffle]
    if self.is_labeled:
      tags = [self.tags[i] for i in shuffle]
    else:
      tags = None

    batched_word_indices = []
    batched_tag_indices = []
    batched_lengths = []

    #############################
    #       YOUR CODE HERE      #
    #############################
    start=0
    remaining = len(self.sentences)
    while(remaining>0):
      #print('remaining', remaining, 'start', start)
      if(int(remaining/batch_size)>=1):
          end = start+batch_size
          batch_sentences = sentences[start:end]
          if(self.is_labeled):
            batch_tags = tags[start:end]
      else:
          batch_sentences = sentences[start:]
          if(self.is_labeled):
            batch_tags = tags[start:]

      max_seq_len = 0
      for sentence in batch_sentences:
        if(len(sentence)>max_seq_len):
          max_seq_len = len(sentence)
      
      word_indices_matrix = PAD_INDEX * np.ones((len(batch_sentences),max_seq_len))

      if(self.is_labeled):
        tag_matrix = IGNORE_TAG_INDEX * np.ones((len(batch_sentences),max_seq_len))
        
      length_matrix = np.zeros((len(batch_sentences),))

      for i,sentence in enumerate(batch_sentences):
        length_matrix[i] = len(sentence)
        for j,word in enumerate(sentence):

          if(word in vocab):
            word_indices_matrix[i,j] = vocab[word]
          else:
            word_indices_matrix[i,j] = UNKNOWN_INDEX
          
          if(self.is_labeled):
            tag_matrix[i,j] = tagset[batch_tags[i][j]]
            
      batched_word_indices.append(word_indices_matrix)
      if(self.is_labeled):
        batched_tag_indices.append(tag_matrix)
      batched_lengths.append(length_matrix)

      start=end
      remaining -= batch_size 

    #############################
    #       DO NOT MODIFY       #
    #############################
    if self.is_labeled:
      return batched_word_indices, batched_tag_indices, batched_lengths
    else:
      return batched_word_indices, batched_lengths

In [0]:
def read_tagset(tag_file):
  """
  Utility function, loads tag file into a dictionary from tag string to tag index

  Arguments:
  - tag_file:   file location of the tagset

  Outputs:
  - tagset:     a dictionary mapping tag strings (e.g. "VB") to a unique index
  """
  tagset = {}
  with open(tag_file, encoding='utf8') as f:
    for line in f:
      columns = line.rstrip().split('\t')
      tag = columns[0]
      tag_id = int(columns[1])
      tagset[tag] = tag_id
  
  return tagset

The cells below download the data files and construct the corresponding `Dataset` objects. 

In [0]:
%%capture
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_4/pos.train
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_4/pos.dev
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_4/pos.test
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_4/pos.tagset

In [0]:
# read the files
tagset = read_tagset('pos.tagset')
train_dataset = Dataset('pos.train', is_labeled=True)
dev_dataset = Dataset('pos.dev', is_labeled=True)
test_dataset = Dataset('pos.test', is_labeled=False)

BATCH_SIZE = 32

# these should run without errors if implemented correctly
train_batch_idx, train_batch_tags, train_batch_lens = train_dataset.get_batches(BATCH_SIZE, vocab, tagset)
dev_batch_idx, dev_batch_tags, dev_batch_lens = dev_dataset.get_batches(BATCH_SIZE, vocab, tagset)
test_batch_idx, test_batch_lens = test_dataset.get_batches(BATCH_SIZE, vocab, tagset)

In [67]:
test_batch_idx[0][0]

array([  1.,  43., 109.,  35.,   6., 328., 139.,   6., 163.,  12.,   9.,
         1., 582.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.])

### Part 2: Evaluation

Next, we will implement utility functions that will later be used to assess our model's perfomance. 

**Please make sure that**

*   Your implementation is self-contained. That is, keep all helper functions or variables inside of your function.
*   Your implementation does not import any additional libraries. You will not receive credit if you do.

In [0]:
# The accuracy function has been implemented for you

def accuracy(true, pred):
  """
  Arguments:
  - true:       a list of true label values (integers)
  - pred:       a list of predicted label values (integers)

  Output:
  - accuracy:   the prediction accuracy
  """
  true = np.array(true)
  pred = np.array(pred)

  num_correct = sum(true == pred)
  num_total = len(true)
  return num_correct / num_total

In [0]:
def confusion_matrix(true, pred, num_tags):
  """
  Arguments:
  - true:       a list of true label values (integers)
  - pred:       a list of predicted label values (integers)
  - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

  Output: 
  - confusion_matrix:   a (num_tags x num_tags) matrix of integers

  confusion_matrix[i][j] = # predictions where true label
  was i and predicted label was j

  """

  confusion_matrix = np.zeros((num_tags, num_tags))

  #############################
  #       YOUR CODE HERE      #
  #############################
  for i in range(len(true)):
    if pred[i]!= -100 or trure[i]!=-100: confusion_matrix[true[i],pred[i]]+=1
  

  return confusion_matrix

In [0]:
def precision(true, pred, num_tags):
  """
  Arguments:
  - true:       a list of true label values (integers)
  - pred:       a list of predicted label values (integers)
  - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

  Output: 
  - precision:  an array of length num_tags, where precision[i]
                gives the precision of class i

  Hints:  the confusion matrix may be useful
          be careful about zero division
  """

  precision = np.zeros(num_tags)

  #############################
  #       YOUR CODE HERE      #
  #############################
  cm = confusion_matrix(true, pred, num_tags)

  for i,_ in enumerate(precision):
    pred_correct = cm[i,i]
    all_pred = np.sum(cm[:,i])
    if(all_pred!=0):
      precision[i] = pred_correct/all_pred

  
  return precision

In [0]:
def recall(true, pred, num_tags):
  """
  Arguments:
  - true:       a list of true label values (integers)
  - pred:       a list of predicted label values (integers)
  - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

  Output: 
  - recall:     an array of length num_tags, where recall[i]
                gives the recall of class i

  Hints:  the confusion matrix may be useful
          be careful about zero division
  """

  """
  YOUR CODE HERE
  """
  recall = np.zeros(num_tags)

  #############################
  #       YOUR CODE HERE      #
  #############################
  cm = confusion_matrix(true, pred, num_tags)
  for i,_ in enumerate(recall):
    pred_correct = cm[i,i]
    all_true = np.sum(cm[i,:])
    if(all_true!=0):
      recall[i] = pred_correct/all_true

  return recall

In [0]:
def f1_score(true, pred, num_tags):
  """
  Arguments:
  - true:       a list of true label values (integers)
  - pred:       a list of predicted label values (integers)
  - num_tags:   the number of possible tags
                true and pred will both contain integers between
                0 and num_tags - 1 (inclusive)

  Output: 
  - f1:         an array of length num_tags, where f1[i]
                gives the recall of class i
  """
  f1 = np.zeros(num_tags)

  #############################
  #       YOUR CODE HERE      #
  #############################
  rec_val = recall(true, pred, num_tags)
  prec_val = precision(true, pred, num_tags)
  for i in range(num_tags):
    if((rec_val[i] + prec_val[i])!=0):
      f1[i]=(2*rec_val[i]*prec_val[i])/(rec_val[i]+prec_val[i])

  return f1

### Part 3: Building the model

Fill in the blanks in `LSTMTagger`'s `__init__` function. If you get stuck, you can reference PyTorch's [torch.nn documentation](https://pytorch.org/docs/stable/nn.html) or [this official tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) on LSTM sequence labeling.

In [0]:
class LSTMTagger(nn.Module):
  """
  An LSTM model for sequence labeling

  Initialization Arguments:
  - embeddings:   a matrix of size (vocab_size, emb_dim)
                  containing pretrained embedding weights
  - hidden_dim:   the LSTM's hidden layer size
  - tagset_size:  the number of possible output tags

  """
  def __init__(self, embeddings, hidden_dim, tagset_size):
    super().__init__()
  
    self.hidden_dim = hidden_dim
    self.num_labels = tagset_size

    #############################
    #       YOUR CODE HERE      #
    #############################
    embedding_dim = embeddings.shape[1]
    # Initialize a PyTorch embeddings layer using the pretrained embedding weights
    self.embeddings = nn.Embedding.from_pretrained(embeddings, freeze=False)

    # Initialize an LSTM layer
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)

    # Initialize a single feedforward layer
    self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
  
  def forward(self, indices, lengths):
    """
    Runs a batched sequence through the model and returns output logits

    Arguments:
    - indices:  a matrix of size (batch_size x max_seq_len)
                containing the word indices of sentences in the batch
    - lengths:  a vector of size (batch_size) containing the
                original lengths of the sequences before padding

    Output:
    - logits:   a matrix of size (batch_size x max_seq_len x num_tags)
                gives a score to each possible tag for each word
                in each sentence 
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # cast arrays as PyTorch data types and move to GPU memory
    indices = torch.LongTensor(indices).to(device)
    lengths = torch.LongTensor(lengths).to(device)
    
    # convert word indices to word embeddings
    embeddings = self.embeddings(indices)

    # pack/pad handles variable length sequence batching
    # see here if you're curious: https://gist.github.com/HarshTrivedi/f4e7293e941b17d19058f6fb90ab0fec
    packed_input_embs = pack_padded_sequence(embeddings, lengths, batch_first=True, enforce_sorted=False)
    # run input through LSTM layer
    packed_output, _ = self.lstm(packed_input_embs)
    # unpack sequences into original format
    padded_output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)

    logits = self.hidden2tag(padded_output)
    return logits

  def run_training(self, train_dataset, dev_dataset, batch_size, vocab, tagset,
                         lr=5e-4, num_epochs=100, eval_every=5):
    """
    Trains the model on the training data with a learning rate of lr
    for num_epochs. Evaluates the model on the dev data eval_every epochs.

    Arguments:
    - train_dataset:  Dataset object containing the training data
    - dev_dataset:    Dataset object containing the dev data
    - batch_size:     batch size for train/dev data
    - vocab:          a dictionary mapping word strings to indices
    - tagset:         a dictionary mapping tag strings to indices
    - lr:             learning rate
    - num_epochs:     number of epochs to train for
    - eval_every:     evaluation is run eval_every epochs
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if str(device) == 'cpu':
      print("Training only supported in GPU environment")
      return

    # clear unreferenced data/models from GPU memory 
    torch.cuda.empty_cache()
    # move model to GPU memory
    self.to(device)

    # set the optimizer (Adam) and loss function (CrossEnt)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss(ignore_index=-100)

    # batch training and dev data
    train_batch_idx, train_batch_tags, train_batch_lens = train_dataset.get_batches(BATCH_SIZE, vocab, tagset)
    dev_batch_idx, dev_batch_tags, dev_batch_lens = dev_dataset.get_batches(BATCH_SIZE, vocab, tagset)

    print("**** TRAINING *****")
    for i in range(num_epochs):
      # sets the model in train mode
      self.train()

      total_loss = 0
      for b in range(len(train_batch_idx)):
        # compute the logits
        logits = model.forward(train_batch_idx[b], train_batch_lens[b])
        # move labels to GPU memory
        labels = torch.LongTensor(train_batch_tags[b]).to(device)
        # compute the loss with respect to true labels
        loss = loss_function(logits.view(-1, len(tagset)), labels.view(-1))
        total_loss += loss
        # propagate gradients backward
        loss.backward()
        optimizer.step()
        # set model gradients to zero before performing next forward pass
        self.zero_grad()

      print("Epoch {} | Loss: {}".format(i, total_loss))

      if (i + 1) % eval_every == 0:
        print("**** EVALUATION *****")
        # sets the model in evaluate mode (no gradients)
        self.eval()
        # compute dev f1 score
        acc, true, pred = self.evaluate(dev_batch_idx, dev_batch_lens, dev_batch_tags, tagset)
        print("Dev Accuracy: {}".format(acc))
        print("**********************")

  def evaluate(self, batched_sentences, batched_lengths, batched_labels, tagset):
    """
    Evaluate the model's predictions on the provided dataset. 

    Arguments:
    - batched_sentences:  a list of matrices, each of size (batch_size x max_seq_len),
                          containing the word indices of sentences in the batch
    - batched_lengths:    a list of vectors, each of size (batch_size), containing the
                          original lengths of the sequences before padding
    - batched_labels:     a list of matrices, each of size (batch_size x max_seq_len),
                          containing the tag indices corresponding to sentences in the batch
    - num_tags:           the number of possible output tags

    Output:
    - accuracy:           the model's prediction accuracy
    - all_true_labels:    a flattened list of all true labels
    - all_predictions:    a flattened list of all of the model's corresponding predictions

    """

    all_true_labels = []
    all_predictions = []

    for b in range(len(batched_sentences)):
      logits = self.forward(batched_sentences[b], batched_lengths[b])
      batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()

      batch_size, _ = batched_sentences[b].shape

      for i in range(batch_size):
        tags = batched_labels[b][i]
        preds = batch_predictions[i]
        
        seq_len = int(batched_lengths[b][i])
        for j in range(seq_len):
          all_predictions.append(int(preds[j]))
          all_true_labels.append(int(tags[j]))
      
    
    acc = accuracy(all_true_labels, all_predictions)
      
    return acc, all_true_labels, all_predictions

In [0]:
def set_seed(seed):
  """
  Sets random seeds and sets model in deterministic
  training mode. Ensures reproducible results
  """
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(seed)

## Training the model

Run the cells below to train your model. If all of the previous sections are implemented correctly, you should see


*   the loss decreasing consistently for every epoch
*   the dev accuracy increasing until convergence around ~0.88

The staff solution achieves an accuracy of 0.880 after 25 epochs.

In [37]:
# sets the random seed – DO NOT change this
# this ensures deterministic results that are comparable with the staff values
set_seed(159)

HIDDEN_SIZE = 64
# intialize a new LSTMTagger model
model = LSTMTagger(embeddings, HIDDEN_SIZE, len(tagset))
# train the model
model.run_training(train_dataset, dev_dataset, BATCH_SIZE, vocab, tagset,   
                   lr=5e-4, num_epochs=25, eval_every=5)

**** TRAINING *****
Epoch 0 | Loss: 999.9423217773438
Epoch 1 | Loss: 442.25811767578125
Epoch 2 | Loss: 275.37127685546875
Epoch 3 | Loss: 212.16627502441406
Epoch 4 | Loss: 181.42295837402344
**** EVALUATION *****
Dev Accuracy: 0.8586200039769338
**********************
Epoch 5 | Loss: 163.11807250976562
Epoch 6 | Loss: 150.5638885498047
Epoch 7 | Loss: 141.14602661132812
Epoch 8 | Loss: 133.68817138671875
Epoch 9 | Loss: 127.54783630371094
**** EVALUATION *****
Dev Accuracy: 0.8754026645456353
**********************
Epoch 10 | Loss: 122.3573989868164
Epoch 11 | Loss: 117.87614440917969
Epoch 12 | Loss: 113.9259033203125
Epoch 13 | Loss: 110.38809967041016
Epoch 14 | Loss: 107.16902923583984
**** EVALUATION *****
Dev Accuracy: 0.8791409823026447
**********************
Epoch 15 | Loss: 104.20204162597656
Epoch 16 | Loss: 101.43944549560547
Epoch 17 | Loss: 98.83808135986328
Epoch 18 | Loss: 96.3730239868164
Epoch 19 | Loss: 94.02762603759766
**** EVALUATION *****
Dev Accuracy: 0.880373

Once the model is trained, run the cells below to print the precision, recall, and $F_1$ score per class.

In [0]:
def eval_per_class(model, dataset, vocab, tagset):
  """
  Prints precision, recall, and F1 for each class in the tagset
  """
  # batch the data
  batched_idx, batched_tags, batched_lens = dev_dataset.get_batches(BATCH_SIZE, vocab, tagset)
  # compute idx --> tag from tag --> idx
  reverse_tagset = {v: k for k,v in tagset.items()}
  # evaluate model on hold-out set
  acc, true, pred = model.evaluate(batched_idx, batched_lens, batched_tags, tagset)
  true = np.array(true)
  pred = np.array(pred)

  pr = precision(true, pred, len(tagset))
  re = recall(true, pred, len(tagset))
  f1 = f1_score(true, pred, len(tagset))

  for idx, tag in reverse_tagset.items():
    print("***********************")
    print("TAG: {}".format(tag))
    num_pred = np.sum(pred == idx)
    num_true = np.sum(true == idx)
    print("({} pred, {} true)".format(num_pred, num_true))

    print("PRECISION: \t{:.3f}".format(pr[idx]))
    print("RECALL: \t{:.3f}".format(re[idx]))
    print("F1 SCORE: \t{:.3f}".format(f1[idx]))

In [39]:
eval_per_class(model, dev_dataset, vocab, tagset)

***********************
TAG: $
(13 pred, 14 true)
PRECISION: 	1.000
RECALL: 	0.929
F1 SCORE: 	0.963
***********************
TAG: ''
(85 pred, 88 true)
PRECISION: 	0.882
RECALL: 	0.852
F1 SCORE: 	0.867
***********************
TAG: ,
(949 pred, 936 true)
PRECISION: 	0.954
RECALL: 	0.967
F1 SCORE: 	0.960
***********************
TAG: -LRB-
(107 pred, 117 true)
PRECISION: 	0.972
RECALL: 	0.889
F1 SCORE: 	0.929
***********************
TAG: -RRB-
(117 pred, 120 true)
PRECISION: 	0.949
RECALL: 	0.925
F1 SCORE: 	0.937
***********************
TAG: .
(1461 pred, 1503 true)
PRECISION: 	0.988
RECALL: 	0.960
F1 SCORE: 	0.974
***********************
TAG: :
(103 pred, 106 true)
PRECISION: 	0.922
RECALL: 	0.896
F1 SCORE: 	0.909
***********************
TAG: ADD
(19 pred, 81 true)
PRECISION: 	0.368
RECALL: 	0.086
F1 SCORE: 	0.140
***********************
TAG: AFX
(1 pred, 4 true)
PRECISION: 	0.000
RECALL: 	0.000
F1 SCORE: 	0.000
***********************
TAG: CC
(780 pred, 781 true)
PRECISION: 	0.990
RECALL

## Part 4: Model Exploration

Congratulations, you've just trained a neural network!

Now, improve the `LSTMTagger` model and implementing the `init` function in the `FancyTagger` class below. 
* Feel free to replace the `forward` function inherited from `LSTMTagger` if 
you need to, but it should not be necessary to receive full credit. Credit will be awarded based on the performance on a holdout test set. 
* Do not modify any of the cells above when completing part 4. Instead, insert cells below if you need to perform any additional computations. 
* You are allowed to use any function in `torch.nn`. You are **not** allowed to import any libraries or use implementations copied from the internet. 

Before submitting, please describe your modifications below:


[ YOUR TEXT HERE ]

In [0]:
class FancyTagger(LSTMTagger):
  """
  An improved neural model for sequence labeling

  Starter code from LSTMTagger has already been provided, but
  feel free to change the init and forward function internals
  if your model design requires it (though this is not necessary
  to receive full credit).

  You may use any component in torch.nn. You may NOT
  import any additional libraries/modules. 

  """
  def __init__(self, embeddings, hidden_dim, tagset_size):
    # initializes the parent LSTMTagger class
    # inherits forward, evaluate, and run_training methods
    super().__init__(embeddings, hidden_dim, tagset_size)
  
    self.hidden_dim = hidden_dim
    self.num_labels = tagset_size

    #############################
    #       YOUR CODE HERE      #
    #############################
    # Initialize a PyTorch embeddings layer using the pretrained embedding weights
    embedding_dim = embeddings.shape[1]
    self.embeddings = nn.Embedding.from_pretrained(embeddings, freeze=False)
    # Initialize an LSTM layer
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers =2, bidirectional=True, dropout=0.5)
    
    # Initialize a single feedforward layer
    self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

In [0]:
# this loads the 10,000 most common word 50-dimensional embeddings
vocab_size = 40000
fancy_embeddings, fancy_vocab = read_embeddings('glove.6B.300d.txt', vocab_size)

Run the training script below to train the `FancyTagger` model. Again, feel free to adjust any hyperparameters if necessary.

In [44]:
model = FancyTagger(fancy_embeddings, HIDDEN_SIZE, len(tagset))
print(model)
model.run_training(train_dataset, dev_dataset, BATCH_SIZE, fancy_vocab, tagset,   
                   lr=5e-4, num_epochs=10, eval_every=5)

FancyTagger(
  (embeddings): Embedding(40000, 300)
  (lstm): LSTM(300, 64, num_layers=2, dropout=0.5, bidirectional=True)
  (hidden2tag): Linear(in_features=128, out_features=50, bias=True)
)
**** TRAINING *****
Epoch 0 | Loss: 712.3302612304688
Epoch 1 | Loss: 193.58493041992188
Epoch 2 | Loss: 123.20848083496094
Epoch 3 | Loss: 96.59712982177734
Epoch 4 | Loss: 81.6850357055664
**** EVALUATION *****
Dev Accuracy: 0.9166036985484192
**********************
Epoch 5 | Loss: 71.25888061523438
Epoch 6 | Loss: 63.308528900146484
Epoch 7 | Loss: 55.8953971862793
Epoch 8 | Loss: 50.27657699584961
Epoch 9 | Loss: 45.72669982910156
**** EVALUATION *****
Dev Accuracy: 0.9206601710081527
**********************


### Save Predictions

When you are satisfied with your `FancyTagger`'s performance on the dev set, run the cell below to write your predictions on the test set to a text file. 

You can download `predictions.txt` by going to 
**View > Table of Contents > Files**

Please submit this `predictions.txt` file to Gradescope. 

In [68]:
assert isinstance(model, FancyTagger), 'Please assign your FancyTagger to a variable named model'

BATCH_SIZE = 32
test_batch_idx, test_batch_lens = test_dataset.get_batches(BATCH_SIZE, fancy_vocab, tagset)

predictions = []

for b in range(len(test_batch_idx)):
  logits = model.forward(test_batch_idx[b], test_batch_lens[b])
  batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()

  batch_size, _ = test_batch_idx[b].shape

  for i in range(batch_size):
    preds = batch_predictions[i]
    print(preds)
    seq_len = int(test_batch_lens[b][i])
    for j in range(seq_len):
      predictions.append(int(preds[j]))
  

with open('predictions.txt', 'w') as f:
  for p in predictions:
    f.write(str(p) + "\n")

[23 29 21 38 36 38 31 36 38 16 11 23 23 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24]
[17 23  2 31 43 16 47 29 42 29  5 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24]
[11 42 41 41 16 17 26 16 24 24 16 24  9 16 24  5 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24]
[16 24 28 26 41 16 23  5 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24]
[11 26 16 24 24 24 39 31 17  9 17 36 38 16  5 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24]
[23 43 31 41 31 38 11 26 11 23  5 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 

In [51]:
test_batch_idx

[array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]]), array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]]), array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]]), array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]]), array([

In [0]:
print(preds)